In [94]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import random
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset
import nltk
from nltk.translate.bleu_score import sentence_bleu

In [95]:
random.seed(42)
device_map = {"":0}

In [96]:
import json 
with open("data/commongen/adversarial/commongen.adversarial.jsonl", "r") as f:
    data = [json.loads(line) for line in f]

data[0]

{'concept_set': 'field#look#stand',
 'scene': ['The player stood in the field looking at the batter.',
  'The coach stands along the field, looking at the goalkeeper.',
  'I stood and looked across the field, peacefully.',
  'Someone stands, looking around the empty field.'],
 'reason': ['A baseball player playing in the field is waiting for the pitcher to pitch the ball to the batter. The player will stand ready while watching the batter.',
  'Soccer is played on a field. Soccer has coaches and goalkeepers.',
  'You have a better view to look at things when you stand. A field is a setting in which one can be.',
  'n/a']}

In [97]:
concept_sets = [d['concept_set'] for d in data]
references_texts = [d['scene'] for d in data]
references_texts[0]

['The player stood in the field looking at the batter.',
 'The coach stands along the field, looking at the goalkeeper.',
 'I stood and looked across the field, peacefully.',
 'Someone stands, looking around the empty field.']

In [98]:
with open("/users/ug21/sachiray/AML/COLD_decoding/data/commongen/adversarial/0f_zx_seed12_0_0_lexical_generation_cw0.500_c2w0.100_lrnllp0.600_len10_topk5_niter2000_frozlen0_winiter1000_noiseiter1_gsstd0.0100_lr0.100_lrratio1.00_lriter1000_50,500,1000,1500_1,0.5,0.1,0.05_output.json") as f:
    pred_data = [json.loads(line) for line in f]

In [99]:
len(pred_data)

6

In [100]:
def calculate_bleu_score(refenrence_sentences, translated_sentence):
    references_tokenized = [nltk.word_tokenize(reference_sentence.lower()) for reference_sentence in refenrence_sentences]
    translated_tokenized = nltk.word_tokenize(translated_sentence.lower())
    bleu_score = sentence_bleu(references_tokenized, translated_tokenized, weights=(1,0,0,0))
    return bleu_score

In [101]:
blue_scores = []
for i in range(len(pred_data)):
    reference_sentences = references_texts[i]
    blue_scores_i = []
    for pred in pred_data[i]['generation_complete']:
        blue_scores_i.append(calculate_bleu_score(reference_sentences, pred))
    blue_scores.append(blue_scores_i)

In [102]:
len(blue_scores), len(blue_scores[0])

(6, 256)

In [103]:
mean_blue = [sum(blue_scores_i)/len(blue_scores_i) for blue_scores_i in blue_scores]
mean_blue

[0.26316407412257725,
 0.30152111940086374,
 0.30902595169758973,
 0.30972827203409603,
 0.37570762612020975,
 0.24226407984102988]

In [104]:
topk_mean = sum(mean_blue)/len(mean_blue)
topk_mean

0.30023518720272774